First we need to import all of the modules we will need.

In [1]:
from nltk import word_tokenize
import nltk
nltk.download('punkt_tab') #for some reason I needed all of these downloaded
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
from datetime import datetime
import json

#Setting a dirname to make the program easier for other people to use
DIRNAME = 'jeopardy.json'
#initializes the stopwords
english_stopwords = set(stopwords.words('english') + list(punctuation) + ['..','...','....','``',"''", '//n'])

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\abiga\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abiga\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abiga\AppData\Roaming\nltk_data...


In [3]:
jepDict = json.load(open(DIRNAME))

I have decided that the questions that are considered "hard" are going to be the higher value questions. The next cell cleans the "value" category to remove the "$" character and the commas on 1,000 and 2,000.

In [7]:
for entry in jepDict:
  values = entry.get('value', '')
  if values:
    cleanVals = values.replace(',','').replace('$', '')
    value = int(cleanVals)
  elif values == 'None':
    value = 0
  entry['value'] = value

Here I am defining a function that will collect just the question and the answer then lemmatize and tokenize those words and exclude the ones that are stopwords.

In [9]:
def clean_wordlist(jsonList):
  '''
  Takes a list of dictionaries, and provides
  a word list for all the questions and answers, lemmatized wihout stopwwords.
  '''
  corpus = ''
  lemmatizer = WordNetLemmatizer()
  for dictionary in jsonList:
    question = dictionary.get('question').lower()
    answer = dictionary.get('answer').lower()
    corpus += '\n'.join([question, answer])
  wordlist = [lemmatizer.lemmatize(x) for x in word_tokenize(corpus) if x not in english_stopwords]
  return wordlist

In [11]:
print(datetime.now().strftime(':%M:%S'))
jepWords = clean_wordlist(jepDict)
print(datetime.now().strftime(':%M:%S')) #Just printing to see how long it takes

:11:29
:13:44


I kept getting an error because my lists were not the same length. Turns out I need to classify each word in worlist first then I can add them to the dataframe. Thanks to chatGPT for helping me figure this out.

In [31]:
hard_text = [entry for entry in jepDict if entry['value'] > 600]
hardWords = set(clean_wordlist(hard_text))

Now to label the words

In [33]:
labels = [ 1 if word in hardWords else 0 for word in jepWords]

In [39]:
df = pd.pandas.DataFrame({'text':jepWords, 'hard':labels})

In [37]:
df

,0,1,2,3,4,5,6,7,8,9,...,2280394,2280395,2280396,2280397,2280398,2280399,2280400,2280401,2280402,2280403
text,'for,last,8,year,life,galileo,house,arrest,espousing,man,...,name,18th,c.,statesman,favorite,catherine,great,grigori,alexandrovich,potemkin
hard,1,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1


Now for the model

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score #This tells us how accurate we were with training our model


In [67]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['hard'], test_size = 0.2)

In [69]:
tfidf_vectorizer = TfidfVectorizer(use_idf = True)
X_train_tf = tfidf_vectorizer.fit_transform(X_train) #chaging th tfidf data into trained vectors
X_test_tf = tfidf_vectorizer.transform(X_test) #vectorizing it bt not the second part

In [71]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_tf, y_train) #try to fit this using our training data
prediction = naive_bayes.predict(X_test_tf) #predict the liklihood for the rest of them

In [73]:
print('Accuracy: ', accuracy_score(y_test, prediction)) #predicts the accuracy of the model

Accuracy:  0.946360522888428


This seems a little bit high, and I felt like the best way to classify hard questions was to use the value that jeapordy already assigned to the question.